In [ ]:
#to detect face and save face
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import time
import os
import cv2 as cv
import os
import face_recognition


location = os.getcwd()
# make a list of all the available images
images = os.listdir('wholefaces')


#put __file__ in quotes
file_path = os.path.dirname(os.path.abspath('__file__')) + os.sep

# objects' confidence threshold
threshold = 0.5  
perfect = .995

#put the appropriate paths in between the quotes provided and the 2 dedicated files 
path = r'\faces'
path1  = r'\wholefaces'

#To load the ResNet model to detect the face
prototxt_file = file_path + 'Resnet_SSD_deploy.prototxt'
caffemodel_file = file_path + 'Res10_300x300_SSD_iter_140000.caffemodel'
net = cv.dnn.readNetFromCaffe(prototxt_file, caffeModel=caffemodel_file)
print('ResNetSSD caffe model loaded successfully')

cap = cv.VideoCapture(0)
time.sleep(1.0)
fps = FPS().start()


size = (int(cap.get(cv.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv.CAP_PROP_FRAME_HEIGHT)))
out_fps = 20 
fourcc = cv.VideoWriter_fourcc(*'mp4v')  
writer = cv.VideoWriter()
out_path = file_path+'test_out'+os.sep+'example.mp4'
writer.open(out_path, fourcc, out_fps, size, True)
j=0
while True:
    _, frame = cap.read()
    origin_h, origin_w = frame.shape[:2]
    blob = cv.dnn.blobFromImage(cv.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

    net.setInput(blob)
    detections = net.forward()

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > threshold:
            bounding_box = detections[0, 0, i, 3:7] * np.array([origin_w, origin_h, origin_w, origin_h])
            x_start, y_start, x_end, y_end = bounding_box.astype('int')

        
            label = '{0:.2f}%'.format(confidence * 100)
         
            cv.rectangle(frame, (x_start, y_start), (x_end, y_end),(0, 0, 255), 2) 
            
            cv.rectangle(frame, (x_start, y_start-18), (x_end, y_start), (0, 0, 255), -1)
            
            cv.putText(frame, label, (x_start+2, y_start-5), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
           
            
            if confidence > perfect:
                text = ("Click")
                cv.putText(frame,text, (15, int(origin_h * 0.5)), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                crop_img = frame[y_start: y_end , x_start: x_end] 
                
           
            
            if cv.waitKey(1) & 0xFF == ord('c'):
                return_value,image = cap.read()
                cv.imwrite(os.path.join(path1,"wholeface"+str(j+1)+".jpg"), image)
                
                # load your image in the space between r" and \wholeface, put the path 1
                image_to_be_matched = face_recognition.load_image_file(r"\wholeface"+str(j+1)+".jpg")
                j=j+1
                # encoded the loaded image into a feature vector
                image_to_be_matched_encoded = face_recognition.face_encodings(image_to_be_matched)[0]
                
                # iterate over each image
                for image in images:
                #load image   
                    current_image = face_recognition.load_image_file("wholefaces/" + image)
                #encode the loaded image into a feature vector
                    current_image_encoded = face_recognition.face_encodings(current_image)[0]
                # match your image with the image and check if it matches
                    result = face_recognition.compare_faces([image_to_be_matched_encoded], current_image_encoded)
                # check if it was a match
                    print(result) 
                if result[0] == False:
                    cv.imwrite(os.path.join(path,"face"+str(j)+".jpg"), crop_img)
                    j=j+1
                
                  
           
    fps.update()
    fps.stop()
    text = "FPS: {:.2f}".format(fps.fps())
    cv.putText(frame, text, (15, int(origin_h * 0.92)), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    cv.imshow('Frame', frame)
    writer.write(frame)

    if cv.waitKey(1) & 0xFF == ord("q"):
        break


writer.release()
cap.release()
cv.destroyAllWindows()